# Git Commit Productivity Analysis (Python Files Only)
This notebook analyzes a local Git repository's commit history to generate developer productivity metrics.

In [ ]:
!pip install gitpython pandas matplotlib

In [ ]:
from git import Repo
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

In [ ]:
# Update with your local repo path
repo_path = "/path/to/your/repo"
repo = Repo(repo_path)

data = []
for commit in repo.iter_commits():
    author = commit.author.email
    date = commit.authored_datetime.date()

insertions = 0
deletions = 0

for fname, stats in commit.stats.files.items():
    insertions += stats['insertions']
    deletions += stats['deletions']

    if insertions > 0 or deletions > 0:
        data.append({
            'author': author,
            'date': pd.to_datetime(date),
            'insertions': insertions,
            'deletions': deletions
        })

df = pd.DataFrame(data)
df.head()

In [ ]:
# Commits per author
commit_counts = df['author'].value_counts()

# Lines added/removed per author
lines_by_author = df.groupby('author')[['insertions', 'deletions']].sum()
lines_by_author['commits'] = df['author'].value_counts()

lines_by_author

In [ ]:
commit_counts.plot(kind='bar', figsize=(8, 4), title='Commits per Developer', ylabel='Commits')
plt.xticks(rotation=45)
plt.grid(True)
plt.show()

In [ ]:
lines_by_author[['insertions', 'deletions']].plot(kind='bar', figsize=(10, 5), title='Lines Added vs Removed by Developer')
plt.ylabel("Lines of Code")
plt.xticks(rotation=45)
plt.grid(True)
plt.show()

In [ ]:
df['month'] = df['date'].dt.to_period('M').dt.to_timestamp()
monthly_summary = df.groupby('month')[['insertions', 'deletions']].sum()
monthly_summary['commits'] = df.groupby('month').size()
monthly_summary

In [ ]:
monthly_summary[['insertions', 'deletions']].plot(figsize=(10, 5), title='Code Changes Over Time (Monthly)', marker='o')
plt.ylabel("Lines of Code")
plt.grid(True)
plt.show()

In [ ]:
monthly_summary['commits'].plot(kind='bar', figsize=(10, 4), title='Commits Per Month')
plt.ylabel("Commits")
plt.xticks(rotation=45)
plt.grid(True)
plt.show()